# Model Registering

In [1]:
# Import required libraries
import numpy as np
import pandas as pd
import os
import shutil 
import requests

import mlflow
import json
import yaml

import warnings
warnings.filterwarnings("ignore")

In [1]:
!conda list dvc

# packages in environment at /opt/anaconda3/envs/food-rec:
#
# Name                    Version                   Build  Channel
dvc                       3.62.0             pyhd8ed1ab_0    conda-forge
dvc-data                  3.16.12            pyhd8ed1ab_0    conda-forge
dvc-http                  2.32.0             pyhd8ed1ab_1    conda-forge
dvc-objects               5.1.1              pyhd8ed1ab_0    conda-forge
dvc-render                1.0.2              pyhd8ed1ab_1    conda-forge
dvc-studio-client         0.22.0             pyhd8ed1ab_0    conda-forge
dvc-task                  0.40.2             pyhd8ed1ab_1    conda-forge


## Preparation

In [3]:
def load_params():
    with open("../params.yaml") as f:
        params = yaml.safe_load(f)
    return params

# Load params
params = load_params()

# Load recipes data
data = pd.read_pickle(os.path.join("../", params["model_pipeline"]["recipe_path"]))

In [4]:
# Load validation model metadata
with open("../validation_metadata.json", "r") as f:
    metadata = json.load(f)

metadata["run_id"]

'570e3720b39f429f8af65a165cf529bb'

In [12]:
mlflow.set_tracking_uri("http://127.0.0.1:8080")
runs = mlflow.search_runs(experiment_names=["FastText-Model"])
sorted_runs = runs.sort_values(by=["start_time", "metrics.RMSE", "metrics.MSE", "metrics.MAE"], ascending=True)
sorted_runs = sorted_runs.sort_values(by=["metrics.Hit-Rate", "metrics.MAP", "metrics.MRR", "metrics.NDCG",  "metrics.F1_10"], ascending=False)
best_run = sorted_runs.iloc[0]
best_run["run_id"]


'03e055bcf6b644d2b44262668e6cb6fd'

In [14]:
runs

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.NDCG,metrics.F1_10,metrics.Precision_10,metrics.RMSE,...,params.sg,params.w_cosine,params.window,params.w_title,params.epochs,tags.mlflow.runName,tags.mlflow.user,tags.mlflow.source.git.commit,tags.mlflow.source.name,tags.mlflow.source.type
0,df7e84eb186c43ddb84f3ae130eeaaf9,977490736678194877,FINISHED,mlflow-artifacts:/977490736678194877/df7e84eb1...,2025-09-09 19:43:21.920000+00:00,2025-09-09 19:43:44.007000+00:00,0.838371,0.425403,0.436111,0.222157,...,0,0.2,8,0.2,100,bold-shrike-679,alfredmastan,9d6ae2b4f17a36f617b392cf2e0bb3b520398ece,src/model_training.py,LOCAL
1,03e055bcf6b644d2b44262668e6cb6fd,977490736678194877,FINISHED,mlflow-artifacts:/977490736678194877/03e055bcf...,2025-09-08 21:39:43.841000+00:00,2025-09-09 19:54:01.954000+00:00,0.958654,0.468504,0.713862,0.199883,...,0,0.2,8,0.2,100,rare-ape-836,alfredmastan,9d6ae2b4f17a36f617b392cf2e0bb3b520398ece,src/model_training.py,LOCAL
2,f3a11d3bbb6d41f7846479d03dec2157,977490736678194877,FINISHED,mlflow-artifacts:/977490736678194877/f3a11d3bb...,2025-09-07 23:48:14.827000+00:00,2025-09-07 23:48:43.220000+00:00,0.824160,0.417197,0.423603,0.213295,...,0,0.2,8,0.2,100,powerful-mouse-556,alfredmastan,b1893b90076c8815cf1017d8759a51987e10f673,src/model_training.py,LOCAL


In [398]:
with open("../registered_metadata.json", "r") as f:
    registered_metadata = json.load(f)
registered_metadata["run_id"]

'29f4ae3f26834dfd939536d0b57838a9'

## Retrieve and Register Best Model

In [ ]:
mlflow.set_tracking_uri("http://127.0.0.1:8080")

# Get the highest performance and oldest run
runs = mlflow.search_runs(experiment_names=["FastText-Model"], order_by=["start_time ASC", "metrics.ndcg DESC", "metrics.rmse DESC", "metrics.f1_10 DESC"])
best_run = runs.iloc[0]

def main():
    # Check if any registered model exists
    if mlflow.search_registered_models():
        # Get current registered model metadata
        with open("../registered_metadata.json", "r") as f:
            registered_metadata = json.load(f)
        
        if best_run["run_id"] == registered_metadata["run_id"]:
            print(f"{'='*20} Current registered model is the best! {'='*20}")
            return

    # Load validation model metadata
    with open("../validation_metadata.json", "r") as f:
        metadata = json.load(f)
    
    # Save registered model metadata
    with open("../registered_metadata.json", "w") as f:
        f.write(json.dumps(metadata, indent=4))
        print("SAVED")
    
    # Register best model to MLFlow
    with mlflow.start_run(run_id=best_run.run_id):
        mlflow.register_model(
            model_uri=metadata["server_uri"],
            name="fasttext_model"
        )

    print(f"{'='*20} New best model registered! {'='*20}")

if __name__ == "__main__":
    main()

==================== Current registered model is the best! ====================


### API Call Test (Run Server Beforehand)

In [1]:
# Save registered model metadata
with open("../registered_metadata.json", "w") as f:
    f.write("test")
    print("SAVED")

SAVED


In [400]:
# params = {"query": ["rice", "beef", "meat"]}
# response = requests.post(f"http://localhost:8000/recommend/", params=params)
# np.asarray(json.loads(response.json()))

### Delete Old Runs and Artifacts (Based on Performance)

In [ ]:
# Get the highest performance and oldest run
runs = mlflow.search_runs(experiment_names=["FastText-Model"])
sorted_runs = runs.sort_values(by=["start_time", "metrics.RMSE", "metrics.MSE", "metrics.MAE"], ascending=True)
sorted_runs = sorted_runs.sort_values(by=["metrics.Hit-Rate", "metrics.MAP", "metrics.MRR", "metrics.NDCG",  "metrics.F1_10"], ascending=False)
sorted_runs

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.MAP,metrics.Precision_10,metrics.MAE,metrics.RMSE,...,params.local_uri,params.vector_size,params.sg,params.w_maxsim,params.w_title,tags.mlflow.user,tags.mlflow.runName,tags.mlflow.source.name,tags.mlflow.source.git.commit,tags.mlflow.source.type
1,0e70656d046543ac94cb337e888cef9b,977490736678194877,FINISHED,mlflow-artifacts:/977490736678194877/0e70656d0...,2025-09-01 21:07:27.757000+00:00,2025-09-01 21:07:50.663000+00:00,0.373175,0.391095,0.215437,0.245744,...,mlflow/mlartifacts/977490736678194877/models/m...,100,0,0.7,0.1,alfredmastan,orderly-hound-550,src/model_training.py,3e53f58337769df908968702c8d386a4620a6a6a,LOCAL
0,2175514552dd426aa8effe61424df1f4,977490736678194877,FINISHED,mlflow-artifacts:/977490736678194877/217551455...,2025-09-01 21:14:47.155000+00:00,2025-09-01 21:15:11.273000+00:00,0.371984,0.390887,0.218263,0.248498,...,mlflow/mlartifacts/977490736678194877/models/m...,100,0,0.7,0.1,alfredmastan,secretive-fowl-352,src/model_training.py,3e53f58337769df908968702c8d386a4620a6a6a,LOCAL
2,86e788289c9a4f0bb16a5660b8f2c27a,977490736678194877,FINISHED,mlflow-artifacts:/977490736678194877/86e788289...,2025-09-01 01:19:58.694000+00:00,2025-09-01 21:24:10.774000+00:00,0.194015,0.349739,0.191978,0.224916,...,mlflow/mlartifacts/977490736678194877/models/m...,100,0,0.7,0.1,alfredmastan,bright-ape-669,src/model_training.py,81a1cccec9a1da8dd0bbf0aa265db835bbbcb077,LOCAL


In [440]:
runs.end_time.dt.tz_convert('America/Los_Angeles')

0   2025-09-01 14:24:10.774000-07:00
1   2025-09-01 14:07:50.663000-07:00
2   2025-09-01 14:15:11.273000-07:00
Name: end_time, dtype: datetime64[ns, America/Los_Angeles]

In [13]:

test = pd.DataFrame({
    "run_id": "b994241c8a714f9695b876f12abc18d1",
    "metrics.RMSE": 0.2354140258218776,
    "metrics.MSE": 0.0562445525510019,
    "metrics.MAE": 0.20520676635658272,
    "metrics.Precision_10": 0.3914480100937988,
    "metrics.Recall_10": 0.4644712668715929,
    "metrics.F1_10": 0.41228315187951176,
    "metrics.NDCG": 0.8027983071293219,
    "metrics.MAP": 0.3742710944026733,
    "metrics.MRR": 0.4083333333333333,
    "metrics.Hit-Rate": 0.4
}, index=[0])
runs = mlflow.search_runs(experiment_names=["FastText-Model"])
test_pd = pd.concat([runs, test], ignore_index=True).sort_values(by=["start_time", "metrics.RMSE"], ascending=True)
test_pd.sort_values(by=["metrics.NDCG",  "metrics.F1_10"], ascending=False)

2025/09/07 15:40:29 WARNING mlflow.tracking.fluent: Cannot retrieve experiment by name FastText-Model


,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.RMSE,metrics.MSE,metrics.MAE,metrics.Precision_10,metrics.Recall_10,metrics.F1_10,metrics.NDCG,metrics.MAP,metrics.MRR,metrics.Hit-Rate
0,b994241c8a714f9695b876f12abc18d1,NaN,NaN,NaN,NaN,NaN,0.235414,0.056245,0.205207,0.391448,0.464471,0.412283,0.802798,0.374271,0.408333,0.4


In [14]:
runs.iloc[params["model_register"]["max_runs"]:]

,run_id,experiment_id,status,artifact_uri,start_time,end_time


In [ ]:
deprecate_runs = sorted_runs.iloc[params["model_register"]["max_runs"]:]

for index, run in deprecate_runs.iterrows():
    # Delete run from MLFlow
    id = run["run_id"]
    mlflow.delete_run(id)

    # Delete run from local file system
    run_path = os.path.join("../mlflow/mlruns", run["experiment_id"], run["run_id"])
    if os.path.exists(run_path):
        print(f"Deleting local run directory: {run_path}")
        shutil.rmtree(run_path)

    # Delete model artifact from local file system
    model_artifact_path = os.path.join(*run["params.local_uri"].split("/")[:-1])
    if os.path.exists(model_artifact_path):
        print(f"Deleting local model artifacts: {model_artifact_path}")
        shutil.rmtree(model_artifact_path)

    # Delete model artifact from local file system
    artifact_path = os.path.join("mlflow/mlartifacts", run["experiment_id"], run["run_id"])
    if os.path.exists(artifact_path):
        print(f"Deleting local artifacts: {artifact_path}")
        shutil.rmtree(artifact_path)

In [ ]:
os.path.join(*sorted_runs.iloc[0]["params.local_uri"].split("/")[:-1])
os.path.exists()

True

In [459]:
sorted_runs

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.MAP,metrics.Precision_10,metrics.MAE,metrics.RMSE,...,params.local_uri,params.vector_size,params.sg,params.w_maxsim,params.w_title,tags.mlflow.user,tags.mlflow.runName,tags.mlflow.source.name,tags.mlflow.source.git.commit,tags.mlflow.source.type
2,86e788289c9a4f0bb16a5660b8f2c27a,977490736678194877,FINISHED,mlflow-artifacts:/977490736678194877/86e788289...,2025-09-01 01:19:58.694000+00:00,2025-09-01 21:24:10.774000+00:00,0.194015,0.349739,0.191978,0.224916,...,mlflow/mlartifacts/977490736678194877/models/m...,100,0,0.7,0.1,alfredmastan,bright-ape-669,src/model_training.py,81a1cccec9a1da8dd0bbf0aa265db835bbbcb077,LOCAL
1,0e70656d046543ac94cb337e888cef9b,977490736678194877,FINISHED,mlflow-artifacts:/977490736678194877/0e70656d0...,2025-09-01 21:07:27.757000+00:00,2025-09-01 21:07:50.663000+00:00,0.373175,0.391095,0.215437,0.245744,...,mlflow/mlartifacts/977490736678194877/models/m...,100,0,0.7,0.1,alfredmastan,orderly-hound-550,src/model_training.py,3e53f58337769df908968702c8d386a4620a6a6a,LOCAL
0,2175514552dd426aa8effe61424df1f4,977490736678194877,FINISHED,mlflow-artifacts:/977490736678194877/217551455...,2025-09-01 21:14:47.155000+00:00,2025-09-01 21:15:11.273000+00:00,0.371984,0.390887,0.218263,0.248498,...,mlflow/mlartifacts/977490736678194877/models/m...,100,0,0.7,0.1,alfredmastan,secretive-fowl-352,src/model_training.py,3e53f58337769df908968702c8d386a4620a6a6a,LOCAL
